In [1]:
import bisect
import copy
import logging

import numpy as np
import rich
import rich.logging
import time
import torch
import transformers
import re


LOGGER = logging.getLogger(__name__)


logging.basicConfig(
    level=logging.DEBUG, 
    format="%(message)s", 
    handlers=[rich.logging.RichHandler(markup=True)]
)


2023-02-14 18:32:33.090205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 18:32:36.585480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/ai.mila.quebec/apps/arch/distro/libffi/3.2.1/lib:/cvmfs/ai.mila.quebec/apps/arch/common/cudnn/11.2-v8.1/lib64:/cvmfs/ai.mila.quebec/apps/arch/common/cudnn/11.2-v8.1/lib:/cvmfs/ai.mila.quebec/apps/arch/common/cuda/11.2/lib64:/cvmfs/ai.mila.quebec/apps/arch/common/nccl/11.2-v2.8/lib:/cvmfs/ai.mila.quebec/apps/arch/distro/openmpi/4.0.4/lib
2023-02-14 18:32:36.586333: W tensorflow/compiler/xla/s

In [2]:
t = transformers.AutoTokenizer.from_pretrained("google/flan-t5-xxl")

[02/14/23 18:32:40] DEBUG    Starting new HTTPS connection (1): huggingface.co:443           ]8;id=43899;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=400915;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py#1003\1003]8;;\

                    DEBUG    https://huggingface.co:443 "HEAD                                 ]8;id=274481;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=82451;file:///home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/urllib3/connectionpool.py#456\456]8;;\
                             /google/flan-t5-xxl/resolve/main/tokenizer_config.json HTTP/1.1"                      
                             200 0                                                                                 

In [65]:
import bisect
def find_lt(a, x):
    'Find rightmost value less than x'
    i = bisect.bisect_left(a, x)
    if i:
        return i - 1
    raise ValueError

def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect.bisect_right(a, x)
    if i:
        return i - 1
    raise ValueError

def find_gt(a, x):
    'Find leftmost value greater than x'
    i = bisect.bisect_right(a, x)
    if i != len(a):
        return i
    raise ValueError

def find_ge(a, x):
    'Find leftmost item greater than or equal to x'
    i = bisect.bisect_left(a, x)
    if i != len(a):
        return i
    raise ValueError

def extract_match_tokens(*, regexes, strings, tokenizer, tokenizer_kwargs=None):
    ########################################################################
    # Preliminary checks and setup
    ########################################################################
    if tokenizer_kwargs is None:
        tokenizer_kwargs = {}
    assert (
        "return_offsets_mapping" not in tokenizer_kwargs or 
        tokenizer_kwargs["return_offsets_mapping"]), (
        "`return_offsets_mapping` is required."
    )

    # compile the regexes in place:
    for i, regex in enumerate(regexes):
        if isinstance(regex, str):
            regexes[i] = re.compile(regex)
        else:
            assert isinstance(regex, re.Pattern), type(regex).mro()

    ########################################################################
    # Main bout
    ########################################################################
    
    # Tokenize
    tok_output = tokenizer(strings, return_offsets_mapping=True, **tokenizer_kwargs)
    tokens = tok_output["input_ids"]
    offsets = tok_output["offset_mapping"]

    # Extract right boundaries
    # if "return_tensors" not in tokenizer_kwargs or not tokenizer_kwargs["return_tensors"]:
    #     assert False
    #     left_boundaries = []
    #     right_boundaries = []
    #     for offset in offsets:
    #         left_local = []
    #         right_local = []
    #         for entry in offset:
    #             left_local.append(entry[0])
    #             right_local.append(entry[1])
    #         left_boundaries.append(left_local)
    #         right_boundaries.append(right_local)
    # elif (
    #     tokenizer_kwargs["return_tensors"] == "pt" or 
    #     tokenizer_kwargs["return_tensors"] == "np"
    # ):

        # left_boundaries = offsets[:, :, 0]
        # right_boundaries = offsets[:, :, 1]

    left_boundaries = []
    right_boundaries = []
    for offset, mask in zip(offsets, tok_output["attention_mask"]):
        left_local = []
        right_local = []
        largest = 0
        for offset_seq, mask_seq in zip(offset, mask):
            if mask_seq != 0:
                if offset_seq[0] > largest:
                    largest = offset_seq[0]
                

                left_local.append(largest)

                if offset_seq[1] > largest:
                    largest = offset_seq[1]

                right_local.append(largest)

        left_boundaries.append(left_local)
        right_boundaries.append(right_local)

    outputs = []
    LOGGER.debug(
        f"[bold blue]offsets:[/]           {offsets}\n"
        f"[bold blue]left_boundaries:[/]   {left_boundaries}\n"
        f"[bold blue]right_boundaries:[/]  {right_boundaries}\n"
        f"[bold blue]pairs:[/]             {[list(zip(a, b)) for a, b in zip(left_boundaries, right_boundaries)]}\n"
    )

    # Extract the matches
    for i, (toks, l_b, r_b, str_, regex) in enumerate(zip(tokens, left_boundaries, right_boundaries, strings, regexes)):
        matches = list(regex.finditer(str_))
        per_str_output = []
        for j, match in enumerate(matches):
            start_char, end_char = match.span()
            lb_right = torch.searchsorted(torch.tensor(l_b), start_char, side="right")
            lb_left  = torch.searchsorted(torch.tensor(l_b), start_char, side="left")
            rb_right = torch.searchsorted(torch.tensor(r_b), end_char,   side="right")
            rb_left  = torch.searchsorted(torch.tensor(r_b), end_char,   side="left")

            start_idx = find_le(l_b, start_char) - 1
            end_idx   = find_le(r_b, end_char)

            LOGGER.debug(
                "\n" +
                "-" * 40 + "\n" +
                f"[bold green]String {i + 1}/{len(strings)} Match {j + 1}/{len(matches)}:[/]\n" +
                "-" * 40 + "\n" +
                f"[bold blue]string match:[/]     `{str_[start_char:end_char]}`" + "\n" +
                f"[bold blue]token match:[/]      `{tokenizer.decode(toks[start_idx:end_idx + 1])}`" + "\n" +
                f"[bold blue]start_char:[/]        {start_char}" + "\n" +
                f"[bold blue]end_char:[/]          {end_char}"   + "\n" +
                f"[bold blue]start_idx:[/]         {start_idx}"  + "\n" +
                f"[bold blue]end_idx:[/]           {end_idx}"    + "\n"+
                f"[bold blue]l_b_right:[/]         {lb_right}"   + "\n"+
                f"[bold blue]l_b_left:[/]          {lb_left}"    + "\n"+
                f"[bold blue]r_b_right:[/]         {rb_right}"   + "\n"+
                f"[bold blue]r_b_left:[/]          {rb_left}"    + "\n" +
                f"[bold blue]l_b:[/]               " + str([(i, int(b)) for i, b in enumerate(l_b)]) + "\n" +
                f"[bold blue]r_b:[/]               " + str([(i, int(b)) for i, b in enumerate(r_b)]) + "\n" +
                f"[bold blue]both boundaries:[/]   " + str([(i, (int(l), int(r))) for i, (l, r) in enumerate(zip(l_b, r_b))]) + "\n" +
                f"[bold blue]tokens:[/]            " + str([(i, tokenizer.decode([t], skip_special_tokens=False)) for i, t in enumerate(toks)]) + "\n" +
                f"[bold blue]token ids:[/]         " + str([(i, int(t)) for i, t in enumerate(toks)]) + "\n" +
                "-" * 40 + "\n"
            )

            per_str_output.append((start_idx, end_idx, tokenizer.decode(toks[start_idx:end_idx + 1])))
        outputs.append(per_str_output)
    return tok_output, outputs

strings = [
    "This is a string with 32362513213 potatoes and 15 apples", 
    "222",
    "222 ",
]

tok_outputs, outputs = extract_match_tokens(
    regexes=[r"\d+"] * len(strings),
    strings=strings, 
    tokenizer=t, 
    tokenizer_kwargs=dict(),
)


# tok_outputs_copy = copy.deepcopy(tok_outputs)
# for i, (tok_output, output) in enumerate(zip(tok_outputs, outputs)):
#     print(f"{output = }")
#     for output_ in output:
#         print(f"{output_ = }")
#         tok_outputs_copy["input_ids"][i][output_[0] : output_[1] + 1] = 0
#     print(t.decode(tok_outputs_copy["input_ids"][i]))



[02/14/23 20:40:35] DEBUG    offsets:           [[(0, 4), (4, 7), (7, 8), (8, 9), (9, 16), (16,    ]8;id=311732;file:///tmp/ipykernel_28035/482120424.py\482120424.py]8;;\:]8;id=169820;file:///tmp/ipykernel_28035/482120424.py#102\102]8;;\
                             21), (21, 24), (24, 26), (26, 28), (28, 31), (31, 33), (33, 42), (42,                 
                             46), (46, 49), (49, 56), (0, 0)], [(0, 1), (0, 3), (0, 0)], [(0, 1),                  
                             (0, 3), (3, 4), (0, 0)]]                                                              
                             left_boundaries:   [[0, 4, 7, 8, 9, 16, 21, 24, 26, 28, 31, 33, 42,                   
                             46, 49, 56], [0, 1, 3], [0, 1, 3, 4]]                                                 
                             right_boundaries:  [[4, 7, 8, 9, 16, 21, 24, 26, 28, 31, 33, 42, 46,                  
                             49, 56, 56], [1, 3, 3], [1, 3, 4, 4]]                                                 
                             pairs:             [[(0, 4), (4, 7), (7, 8), (8, 9), (9, 16), (16,                    
                             21), (21, 24), (24, 26), (26, 28), (28, 31), (31, 33), (33, 42), (42,                 
                             46), (46, 49), (49, 56), (56, 56)], [(0, 1), (1, 3), (3, 3)], [(0,                    
                             1), (1, 3), (3, 4), (4, 4)]]                                                          
                                                                                                                   

                    DEBUG                                                                          ]8;id=528511;file:///tmp/ipykernel_28035/482120424.py\482120424.py]8;;\:]8;id=244999;file:///tmp/ipykernel_28035/482120424.py#123\123]8;;\
                             ----------------------------------------                                              
                             String 1/3 Match 1/2:                                                                 
                             ----------------------------------------                                              
                             string match:     `32362513213`                                                       
                             token match:      `32362513213`                                                       
                             start_char:        22                                                                 
                             end_char:          33                                                                 
                             start_idx:         6                                                                  
                             end_idx:           10                                                                 
                             l_b_right:         7                                                                  
                             l_b_left:          7                                                                  
                             r_b_right:         11                                                                 
                             r_b_left:          10                                                                 
                             l_b:               [(0, 0), (1, 4), (2, 7), (3, 8), (4, 9), (5, 16),                  
                             (6, 21), (7, 24), (8, 26), (9, 28), (10, 31), (11, 33), (12, 42),                     
                             (13, 46), (14, 49), (15, 56)]                                                         
                             r_b:               [(0, 4), (1, 7), (2, 8), (3, 9), (4, 16), (5, 21),                 
                             (6, 24), (7, 26), (8, 28), (9, 31), (10, 33), (11, 42), (12, 46),                     
                             (13, 49), (14, 56), (15, 56)]                                                         
                             both boundaries:   [(0, (0, 4)), (1, (4, 7)), (2, (7, 8)), (3, (8,                    
                             9)), (4, (9, 16)), (5, (16, 21)), (6, (21, 24)), (7, (24, 26)), (8,                   
                             (26, 28)), (9, (28, 31)), (10, (31, 33)), (11, (33, 42)), (12, (42,                   
                             46)), (13, (46, 49)), (14, (49, 56)), (15, (56, 56))]                                 
                             tokens:            [(0, 'This'), (1, 'is'), (2, ''), (3, 'a'), (4,                    
                             'string'), (5, 'with'), (6, '32'), (7, '36'), (8, '25'), (9, '132'),                  
                             (10, '13'), (11, 'potatoes'), (12, 'and'), (13, '15'), (14,                           
                             'apples'), (15, '</s>')]                                                              
                             token ids:         [(0, 100), (1, 19), (2, 3), (3, 9), (4, 6108), (5,                 
                             28), (6, 3538), (7, 3420), (8, 1828), (9, 23757), (10, 2368), (11,                    
                             11076), (12, 11), (13, 627), (14, 16981), (15, 1)]                                    
                             ----------------------------------------                                              
                                                                                                                   

                    DEBUG                                                                          ]8;id=696021;file:///tmp/ipykernel_28035/482120424.py\482120424.py]8;;\:]8;id=343410;file:///tmp/ipykernel_28035/482120424.py#123\123]8;;\
                             ----------------------------------------                                              
                             String 1/3 Match 2/2:                                                                 
                             ----------------------------------------                                              
                             string match:     `15`                                                                
                             token match:      `15`                                                                
                             start_char:        47                                                                 
                             end_char:          49                                                                 
                             start_idx:         13                                                                 
                             end_idx:           13                                                                 
                             l_b_right:         14                                                                 
                             l_b_left:          14                                                                 
                             r_b_right:         14                                                                 
                             r_b_left:          13                                                                 
                             l_b:               [(0, 0), (1, 4), (2, 7), (3, 8), (4, 9), (5, 16),                  
                             (6, 21), (7, 24), (8, 26), (9, 28), (10, 31), (11, 33), (12, 42),                     
                             (13, 46), (14, 49), (15, 56)]                                                         
                             r_b:               [(0, 4), (1, 7), (2, 8), (3, 9), (4, 16), (5, 21),                 
                             (6, 24), (7, 26), (8, 28), (9, 31), (10, 33), (11, 42), (12, 46),                     
                             (13, 49), (14, 56), (15, 56)]                                                         
                             both boundaries:   [(0, (0, 4)), (1, (4, 7)), (2, (7, 8)), (3, (8,                    
                             9)), (4, (9, 16)), (5, (16, 21)), (6, (21, 24)), (7, (24, 26)), (8,                   
                             (26, 28)), (9, (28, 31)), (10, (31, 33)), (11, (33, 42)), (12, (42,                   
                             46)), (13, (46, 49)), (14, (49, 56)), (15, (56, 56))]                                 
                             tokens:            [(0, 'This'), (1, 'is'), (2, ''), (3, 'a'), (4,                    
                             'string'), (5, 'with'), (6, '32'), (7, '36'), (8, '25'), (9, '132'),                  
                             (10, '13'), (11, 'potatoes'), (12, 'and'), (13, '15'), (14,                           
                             'apples'), (15, '</s>')]                                                              
                             token ids:         [(0, 100), (1, 19), (2, 3), (3, 9), (4, 6108), (5,                 
                             28), (6, 3538), (7, 3420), (8, 1828), (9, 23757), (10, 2368), (11,                    
                             11076), (12, 11), (13, 627), (14, 16981), (15, 1)]                                    
                             ----------------------------------------                                              
                                                                                                                   

                    DEBUG                                                                          ]8;id=723376;file:///tmp/ipykernel_28035/482120424.py\482120424.py]8;;\:]8;id=820672;file:///tmp/ipykernel_28035/482120424.py#123\123]8;;\
                             ----------------------------------------                                              
                             String 2/3 Match 1/1:                                                                 
                             ----------------------------------------                                              
                             string match:     `222`                                                               
                             token match:      `</s>`                                                              
                             start_char:        0                                                                  
                             end_char:          3                                                                  
                             start_idx:         -1                                                                 
                             end_idx:           2                                                                  
                             l_b_right:         1                                                                  
                             l_b_left:          0                                                                  
                             r_b_right:         3                                                                  
                             r_b_left:          1                                                                  
                             l_b:               [(0, 0), (1, 1), (2, 3)]                                           
                             r_b:               [(0, 1), (1, 3), (2, 3)]                                           
                             both boundaries:   [(0, (0, 1)), (1, (1, 3)), (2, (3, 3))]                            
                             tokens:            [(0, ''), (1, '222'), (2, '</s>')]                                 
                             token ids:         [(0, 3), (1, 26144), (2, 1)]                                       
                             ----------------------------------------                                              
                                                                                                                   

                    DEBUG                                                                          ]8;id=930606;file:///tmp/ipykernel_28035/482120424.py\482120424.py]8;;\:]8;id=48704;file:///tmp/ipykernel_28035/482120424.py#123\123]8;;\
                             ----------------------------------------                                              
                             String 3/3 Match 1/1:                                                                 
                             ----------------------------------------                                              
                             string match:     `222`                                                               
                             token match:      ``                                                                  
                             start_char:        0                                                                  
                             end_char:          3                                                                  
                             start_idx:         -1                                                                 
                             end_idx:           1                                                                  
                             l_b_right:         1                                                                  
                             l_b_left:          0                                                                  
                             r_b_right:         2                                                                  
                             r_b_left:          1                                                                  
                             l_b:               [(0, 0), (1, 1), (2, 3), (3, 4)]                                   
                             r_b:               [(0, 1), (1, 3), (2, 4), (3, 4)]                                   
                             both boundaries:   [(0, (0, 1)), (1, (1, 3)), (2, (3, 4)), (3, (4,                    
                             4))]                                                                                  
                             tokens:            [(0, ''), (1, '222'), (2, ''), (3, '</s>')]                        
                             token ids:         [(0, 3), (1, 26144), (2, 3), (3, 1)]                               
                             ----------------------------------------                                              
                                                                                                                   